## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [25]:
import os
import pathlib

In [26]:
this_dir = pathlib.Path(os.path.abspath(""))

In [27]:
data_dir = this_dir / "data"

In [28]:
import re
import pytz
import glob
import requests
import unicodedata
import json
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

### Download / retrieve the page

Lake County's [dashboard](http://health.co.lake.ca.us/Coronavirus/COVID-19_Data.htm) uses Datawrapper to display cases by ZIP Code. To make sure we get the latest, we'll hit the chart page on Datawrapper's website, then get the iframe URL.

In [29]:
url = "https://www.datawrapper.de/_/D1RaA/"

In [30]:
page = requests.get(url)

### Parse with BeautifulSoup

In [31]:
soup = BeautifulSoup(page.content, "html.parser")

In [32]:
meta = soup.find("meta", {"property": "og:url"})

In [33]:
dw_url = meta["content"]

### Parse Datawrapper

In [34]:
dataset_url = dw_url + "dataset.csv"

In [35]:
dw_r = requests.get(dataset_url)

In [36]:
string = dw_r.text

In [37]:
parsed = string.split("\r\n")

In [38]:
data_list = []
for row in parsed:
    cell = row.split(",")
    data_list.append(cell)

In [39]:
data_list

[['Supervisorial District', 'District Total', 'Zip Code', 'Zip Code Total'],
 ['District 1', '863', '95457', '290'],
 ['', '', '95467', '387'],
 ['', '', '95461', '186'],
 ['District 2', '1844', '95422', '1819'],
 ['', '', '95424', '25'],
 ['District 3', '1183', '95423', '274'],
 ['', '', '95458', '290'],
 ['', '', '95464', '278'],
 ['', '', '95485', '315'],
 ['', '', '95493', '12'],
 ['', '', '95443', '14'],
 ['District 4', '1500', '95453', '1452'],
 ['', '', '95435', '48'],
 ['District 5', '1240', '95426', '72'],
 ['', '', '95451', '1168'],
 ['Unknown or Other', '97', 'Unknown/other', '97']]

In [40]:
df = pd.DataFrame(data_list[1:], columns=data_list[0])

### Get timestamp

In [41]:
df["county_date"] = datetime.now().date()

In [42]:
df["county"] = "Lake"

### Rename and drop columns

In [43]:
cols = {"Zip Code": "area", "Zip Code Total": "confirmed_cases"}

In [44]:
export_df = df.rename(columns=cols)[
    ["county", "area", "confirmed_cases", "county_date"]
]

In [45]:
len(export_df)

16

### Vet

Lake County has 16 ZIP code areas, including "Unknown"

In [46]:
lake_len = 16

In [47]:
try:
    assert not len(export_df) > lake_len
except AssertionError:
    raise AssertionError(
        "Lake County scraper: spreadsheet has additional zip code(s) than previous"
    )

In [48]:
try:
    assert not len(export_df) < lake_len
except AssertionError:
    raise AssertionError("Lake County scraper: spreadsheet is missing row(s)")

### Export csv file

Write csv file timestamped with today's date

In [49]:
slug = "lake"

In [50]:
tz = pytz.timezone("America/Los_Angeles")

In [51]:
today = datetime.now(tz).date()

In [52]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [53]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [54]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [55]:
df = pd.concat(df_list).sort_values(["date", "area"]).drop_duplicates()

In [56]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)